In [1]:
from qreduce.cs_vqe import cs_vqe_model
from qreduce.utils import *
from itertools import combinations
import json
import numpy as np

In [2]:
with open('data/model_data.json', 'r') as infile:
        mol_data = json.load(infile)
mol_data.keys()

dict_keys(['H2O_STO-3G_SINGLET', 'LiH_STO-3G_SINGLET', 'HF_STO-3G_SINGLET', 'CH2_STO-3G_SINGLET', 'BeH2_STO-3G_SINGLET', 'HeH+_6-311G_SINGLET', 'BeH+_STO-3G_SINGLET', 'CH+_STO-3G_SINGLET', 'B+_STO-3G_SINGLET', 'OH+_STO-3G_SINGLET', 'OH-_STO-3G_SINGLET', 'O_STO-3G_SINGLET', 'Be_STO-3G_SINGLET', 'H4_STO-3G_SINGLET', 'B_STO-3G_DOUBLET', 'NH_STO-3G_SINGLET', 'BH_STO-3G_SINGLET', 'NH2+_STO-3G_SINGLET', 'BH2+_STO-3G_SINGLET', 'CH2_STO-3G_TRIPLET', 'C_STO-3G_TRIPLET'])

In [3]:
mol=mol_data['Be_STO-3G_SINGLET']
ham=mol['ham']
n_q=mol['num_qubits']
set_nc = mol['terms_noncon']
cs_vqe_mol = cs_vqe_model(ham, noncontextual_set = set_nc)

In [4]:
print("Generators:", cs_vqe_mol.generators)
print("Clique reps:", cs_vqe_mol.cliquereps)
#match=mol['X_index']==cs_vqe_mol.anti_clique_qubit
#print('Anti-clique qubit match?', match, '\n')

print("G assignment:", cs_vqe_mol.nu)#, ' | original:', np.array(list(mol['G'].values()), dtype=int))
print("r vector:", cs_vqe_mol.r)
print("Noncon nrg:", cs_vqe_mol.ngs_energy)
print('NGS matches original?', abs(cs_vqe_mol.ngs_energy-mol['noncon'])<1e-13, '\n')

Generators: ['ZZZZZ', 'ZZZIZ', 'ZIZIZ', 'ZZIIZ', 'ZZZII']
Clique reps: ['ZZZXZ', 'ZZZZZ']
G assignment: [ 1 -1 -1 -1  1]
r vector: [-2.33081609e-08  1.00000000e+00]
Noncon nrg: -14.351880476201996
NGS matches original? True 



In [5]:
num_sim_q = mol['chem_acc_num_q']
exact = exact_gs_energy(cs_vqe_mol.hamiltonian)
print('Exact energy:',exact)
print('Noncon error:', cs_vqe_mol.ngs_energy-exact)
print(f'Target_error for {num_sim_q} qubits:', exact_gs_energy(mol['ham_reduced'][num_sim_q])-exact)

Exact energy: -14.40365510806765
Noncon error: 0.05177463186565312
Target_error for 3 qubits: 0.0003265060781139084


In [6]:
num_sim_q = 3

for order in combinations(range(len(cs_vqe_mol.generators)), cs_vqe_mol.num_qubits - num_sim_q):
    order = list(order)
    ham_cs = cs_vqe_mol.contextual_subspace_hamiltonian(order)
    cs_energy = exact_gs_energy(ham_cs)
    in_range = cs_energy<cs_vqe_mol.ngs_energy and cs_energy>exact
    print(f'CS-VQE error for {num_sim_q} qubits is {cs_energy-exact: .8f}, projecting stabilizers {order}')

CS-VQE error for 3 qubits is  0.05148601, projecting stabilizers [0, 1]
CS-VQE error for 3 qubits is  0.05148386, projecting stabilizers [0, 2]
CS-VQE error for 3 qubits is  0.05148386, projecting stabilizers [0, 3]
CS-VQE error for 3 qubits is  0.05177463, projecting stabilizers [0, 4]
CS-VQE error for 3 qubits is  0.01411851, projecting stabilizers [1, 2]
CS-VQE error for 3 qubits is  0.01411851, projecting stabilizers [1, 3]
CS-VQE error for 3 qubits is  0.05177463, projecting stabilizers [1, 4]
CS-VQE error for 3 qubits is  0.03081720, projecting stabilizers [2, 3]
CS-VQE error for 3 qubits is  0.05177463, projecting stabilizers [2, 4]
CS-VQE error for 3 qubits is  0.05177463, projecting stabilizers [3, 4]
